In [75]:
import cv2
import numpy as np


def draw_tracks(sequence : np.array, tracks_data : np.array) -> np.array:
    '''
    Function that draw the tracks given the sequence and the array of tracks
    
    Input:
        - squence: array of shape (N,H,W,C)
            N: number of frames
            H: Height
            W: Width
            C: Channel (empty if grayscale)
        - tracks_data: array where:
                            tracks_data[0] track_id
                            tracks_data[1] x
                            tracks_data[2] y
                            tracks_data[3] fluorescence
                            tracks_data[4] frame
    
    Ouput:
        - seq_out: array of shape (N,H,W,C) 
    '''
    frames = sequence.shape[0]
    tracks_id = (np.unique(tracks_data[:, 0])).tolist()
    seq_out = sequence.copy()
    
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,500)
    fontScale              = 0.7
    fontColor              = (255,255,255)
    lineType               = 2

    
    tracks = []
    for track_id in tracks_id:
        data_track = tracks_data[tracks_data[:, 0]==track_id]
        coord = np.ones((frames, 3)) * -1
        coord[np.uint(data_track[:, 4])] = data_track[:, 1:4]
        tracks.append(coord)
    colors = np.random.uniform(0, 255, (len(tracks_id), 3))
    
    for frame in range(frames):
        for i in range(1,frame+1):
            for tr, track in enumerate(tracks):
                if track[i,0]>0 and track[i-1,0]>0:
                    cv2.line(seq_out[frame], (int(track[i-1,0]),int(track[i-1,1]) ), (int(track[i,0]),int(track[i,1]) ), colors[tr])
            
                    fontColor = colors[tr]
                    coord = (int(track[frame,0]) + 5,int(track[frame,1]) + 5)
                    cv2.putText(seq_out[frame],'F = %d' % track[frame,2], 
                            coord, 
                            font, 
                            fontScale,
                            fontColor,
                            lineType)
                    
    return np.uint8(seq_out)


In [74]:
import pandas as pd
import tifffile
from imageio import mimwrite as mp4_writer

in_sequence = tifffile.TiffFile("seq.tiff")
sequence = in_sequence.asarray()
data = pd.read_csv("seq_data.csv")
data_in = (data.to_numpy())[:, 0:5]

seq_tracks = draw_tracks(sequence, data_in)
print(seq_tracks.shape)
mp4_writer("output.mp4", seq_tracks, format="mp4", fps=5.)


(100, 512, 512, 3)


In [55]:
data

,id_particle,x,y,frame,intensity,fluorescence
0,0.0,264.517758,27.023621,0.0,NaN,221.074744
1,5.0,245.187939,404.346647,0.0,NaN,153.457489
2,6.0,183.180628,348.638459,0.0,NaN,115.938727
3,11.0,451.748110,103.241481,0.0,NaN,240.745754
4,19.0,27.657664,393.386608,0.0,NaN,145.557898
...,...,...,...,...,...,...
974,9833.0,189.804424,326.050724,96.0,NaN,151.703137
975,9833.0,182.181934,334.930520,97.0,NaN,126.305519
976,11497.0,304.156919,383.335344,97.0,NaN,5.224558
977,9833.0,173.212722,342.786866,98.0,NaN,114.291056
